In [ ]:
import os
import arcpy
from PIL import Image
import numpy as np

In [ ]:
folder = r"" # Folder containing the Spacenet dataset

In [ ]:
# Select folder with geojson footprints
json_folder = os.path.join(folder,"geojson")
arcpy.env.workspace = foljson_folderder
files = [os.path.join(json_folder,file) for file in os.listdir(json_folder)]
files

In [ ]:
# move worskpace to new folder and convert geojsons to shapefiles
arcpy.env.workspace = os.path.join(folder,"shapefiles")
arcpy.env.addOutputsToMap = False
for path in files:
    try:
        start = path.rfind("_")
        name = path[start+1:-8]
        arcpy.conversion.JSONToFeatures(path, name, "POLYGON")
    except:
        pass

In [ ]:
# Merge the shapefiles into single feature class
arcpy.env.workspace = os.path.join(folder,"labels")
shapes = arcpy.ListFeatureClasses()
print(shapes)
arcpy.management.Merge(shapes,"oc_merged")

In [ ]:
# Create a mosaic dataset and add the image rasters

In [ ]:
# Using the DL tools, chip images and labels

In [ ]:
images_folder = os.path.join(folder,"3band")
labels_folder = os.path.join(folder,"shapefiles")
out_folder = os.path.join(folder,"original_resolution")
arcpy.env.workspace = images_folder
image_files = arcpy.ListRasters()

for file in image_files[:100]:
    path = os.path.splitext(file)[0]
    number = path[path.find('img')+3:]
    try:
        img_raster = arcpy.Raster(file)
        lab_shape = os.path.join(labels_folder,f"img{number}.shp")
        name = os.path.join(os.path.join(folder,"Default.gdb"),
                         f"img{number}_PolygonToRaster")
        with arcpy.EnvManager(snapRaster=img_raster):
            arcpy.conversion.PolygonToRaster(lab_shape, "uid", name, "MAXIMUM_AREA", "NONE", img_raster)
        lab_arr = arcpy.RasterToNumPyArray(lab_raster)
        lab_arr = lab_arr.astype('uint8') 
        lab = Image.fromarray(lab_arr)
        lab.save(os.path.join(out_folder,"labels",f"label{number}.jpg"))
        img_arr = arcpy.RasterToNumPyArray(img_raster)
        img_arr = np.moveaxis(img_arr,0,2)
        img = Image.fromarray(img_arr)
        img.save(os.path.join(out_folder,"images",f"image{number}.jpg"))
    except:
        continue